<a href="https://colab.research.google.com/github/VivianYueh/Imformation-Retrieval-Final/blob/main/%E9%97%9C%E9%8D%B5%E5%AD%97%E6%93%B7%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>此處是使用處理中文句子斷詞的範例，包括CKIPtagger及Distiltag</h3>

In [2]:
!pip install -U DistilTag

  Using cached DistilTag-0.2.2-py3-none-any.whl.metadata (390 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [3]:
import DistilTag
DistilTag.download()

Downloading...
From (original): https://drive.google.com/uc?id=1AzUICPQ5MMt_IWg4JZ3mWM6vGbQkv01L
From (redirected): https://drive.google.com/uc?id=1AzUICPQ5MMt_IWg4JZ3mWM6vGbQkv01L&confirm=t&uuid=844a4526-217c-47f9-b9f9-dd92981e6851
To: /tmp/tmpsam65fw6distiltag/tagmodel.zip
100%|██████████| 501M/501M [00:10<00:00, 46.6MB/s]


setting up model...
DistilTag model installed.


In [4]:
from DistilTag import DistilTag
tagger = DistilTag()

### 保留名詞、動詞

In [22]:
def filter_nouns_verbs(tagged):
    """
    過濾出名詞(N*)與動詞(V*)的詞性標註
    """
    return [(word, pos) for word, pos in tagged if pos.startswith('N') or pos.startswith('V')]


### 擷取關鍵字

In [26]:
def unique_words(words):
    return list(dict.fromkeys(words))  # 去重同時保留原順序

In [27]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://www.niar.org.tw'
news_base_url = 'https://www.niar.org.tw/xmdoc?xsmsid=0I148622737263495777'

page = 1
news_count = 0

print("國家實驗研究院新聞列表：\n")

while news_count<10:
    url = f"{news_base_url}&page={page}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    found_news = False

    for row in rows:
        date_td = row.find('td', class_='date')
        title_td = row.find('td', class_='title')

        if date_td and title_td:
            found_news = True
            news_count += 1
            date = date_td.text.strip()
            a_tag = title_td.find('a')
            title = a_tag.text.strip()
            tags = tagger.tag(title)  # 這裡可能是 list[list[tuple]]
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]
            unique = unique_words(words)
            link = base_url + a_tag['href']
            print(f"{news_count}. [{date}] {title}\n   {tags}\n   {unique}\n   👉 {link}\n")

    if not found_news:
        break  # 沒有新內容就停止
    page += 1

國家實驗研究院新聞列表：

1. [2025-04-16] 科教館「科學家的秘密基地」 4/16更新展區重新亮相
   [[('科教館', 'Nc'), ('「', 'PARENTHESISCATEGORY'), ('科學家', 'Na'), ('的', 'DE'), ('秘密', 'Na'), ('基地', 'Nc'), ('」', 'PARENTHESISCATEGORY'), ('４／１６', 'Nd'), ('更新', 'VC'), ('展區', 'Nc'), ('重新', 'D'), ('亮相', 'VA')]]
   ['科教館', '科學家', '秘密', '基地', '４／１６', '更新', '展區', '亮相']
   👉 https://www.niar.org.tw/xmdoc/cont?xsmsid=0I148622737263495777&sid=0P106529706322578269

2. [2025-04-14] 【國研院說明稿】關於小油坑失火事件  若鑑定結果為國研院設備造成 國研院絕不卸責
   [[('【', 'PARENTHESISCATEGORY'), ('國研院', 'Nc'), ('說明稿', 'Na'), ('】', 'PARENTHESISCATEGORY'), ('關於', 'P'), ('小油坑', 'Nc'), ('失火', 'VH'), ('事件', 'Na'), ('若', 'Cbb'), ('鑑定', 'VC'), ('結果', 'Na'), ('為', 'P'), ('國研院', 'Nc'), ('設備', 'Na'), ('造成', 'VG'), ('國研院', 'Nc'), ('絕不', 'D'), ('卸責', 'VA')]]
   ['國研院', '說明稿', '小油坑', '失火', '事件', '鑑定', '結果', '設備', '造成', '卸責']
   👉 https://www.niar.org.tw/xmdoc/cont?xsmsid=0I148622737263495777&sid=0P104744411632087760

3. [2025-04-14] 【國研院說明稿】關於小油坑失火事件
   [[('【', 'PARENTHESISCATEGORY'), ('國研院',

### 內文擷取

In [37]:
import requests
from bs4 import BeautifulSoup
import textwrap

base_url = 'https://www.niar.org.tw'
news_base_url = 'https://www.niar.org.tw/xmdoc?xsmsid=0I148622737263495777'

page = 1
news_count = 0

print("國家實驗研究院新聞列表：\n")

while news_count<10:
    url = f"{news_base_url}&page={page}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    found_news = False

    for row in rows:
        date_td = row.find('td', class_='date')
        title_td = row.find('td', class_='title')

        if date_td and title_td:
            found_news = True
            news_count += 1
            #date = date_td.text.strip()
            a_tag = title_td.find('a')
            title = a_tag.text.strip()
            '''tags = tagger.tag(title)  # 這裡可能是 list[list[tuple]]
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]
            unique = unique_words(words)'''
            link = base_url + a_tag['href']
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # 抓取文章內容
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content = article_soup.find_all('p', class_='MsoNormal')
            content_text = "\n".join(textwrap.fill(p.get_text(strip=True), width=80)  for p in content  if p.get_text(strip=True))
            print(f"{news_count}. {title}\n   {content_text}\n")
            #print(f"{news_count}. [{date}] {title}\n   {tags}\n   {unique}\n   👉 {link}\n")

    if not found_news:
        break  # 沒有新內容就停止
    page += 1

國家實驗研究院新聞列表：

1. 科教館「科學家的秘密基地」 4/16更新展區重新亮相
   國科會轄下之國家實驗研究院及國家太空中心，與國立臺灣科學教育館合作，辦理「科學家的秘密基地」長期展，自2023年3月開展以來，備受各界好評，已吸引超過13萬人入
場參觀。今（4/16）日於更新部分展品後重新開展，希望能幫助參觀民眾透過互動遊戲、親眼觀察與模型展示，習得有趣的科學知識，同時深入認識科研工作。「科學家的秘密基
地」位於科教館8樓東南側扇形展場，分為「實驗基地」、「探測基地」和「智慧基地」三區，每一區均包含兩大展示主題。此次更新的是國家生物模式中心（原國家實驗動物中心）
、國家地震工程研究中心（國震中心）及台灣海洋科技研究中心（海洋中心）的展品。一進門就看到實驗基地－生物模式中心展出的「貓咪毛色變化的秘密」，這裡展示多基因協同作
用的觀念。生物模式中心近年以革新的基因編輯工具「CRISPR/Cas」基因剪刀產製「擬人鼠」動物模式，推進我國精準醫療研究與臨床應用，而科學家進行基因編輯時，須
嚴加考慮生物體複雜的基因層次。現場除了深入淺出的科學觀念，也設計了一個基因密碼轉盤遊戲，參觀者可以透過簡單的科普工具，輕鬆學習生物醫學知識。實驗基地－國震中心透
過互動展示，介紹房屋的共振原理，包括不同樓高建築物的振動特性，以及地震波在不同地質條件下傳遞所產生的共振效應。現場提供以不同長度的吸管模擬各種樓高的共振教具，讓
參觀者能親身體驗結構與地震波共振的現象。同時展示「滾動式隔震平台」，結合電動振動台模擬真實地震波，可清楚觀察放置於隔震裝置上的設備，其振動幅度大幅減少，讓參觀者
直觀感受隔震技術的保護效果。接著來到探測基地－海洋中心，這裡展出於臺灣西南海域及南海採集到的海洋浮游動物標本、寫真、採集工具，以及使用海洋中心自製底碇平台記錄之
水下300米的海底世界。海洋浮游動物在海洋生態系中扮演著重要而多樣的角色，牠們的種類及數量非常容易受到海洋環境的影響，當環境變動時（如溫度、鹽度、酸鹼度），浮游
動物的組成也會隨之改變，間接對魚類、鯨類等各種海洋動物產生連鎖反應。因此浮游動物是一種很好的環境指標生物，可以用來監測海洋環境變遷、預測海洋生態系的改變。展場設
置顯微觀察區，讓參觀者一窺這群嬌小但至關重要的小小兵身影，了解牠們是維護海洋生態平衡的關鍵多數。除了更新的三件展品外，探測基

In [47]:
import requests
from bs4 import BeautifulSoup
import textwrap
from collections import Counter

base_url = 'https://www.niar.org.tw'
news_base_url = 'https://www.niar.org.tw/xmdoc?xsmsid=0I148622737263495777'

page = 1
news_count = 0

special_terms = [
    "國家生物模式中心",
    "國家地震工程研究中心",
    "國家高速網路與計算中心",
    "台灣半導體研究中心",
    "國家儀器科技研究中心",
    "科技政策研究與資訊中心",
    "台灣海洋科技研究中心"
]

print("國家實驗研究院新聞列表：\n")

while news_count<10:
    url = f"{news_base_url}&page={page}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    found_news = False

    for row in rows:
        date_td = row.find('td', class_='date')
        title_td = row.find('td', class_='title')

        if date_td and title_td:
            found_news = True
            news_count += 1
            #date = date_td.text.strip()
            a_tag = title_td.find('a')
            title = a_tag.text.strip()
            '''tags = tagger.tag(title)  # 這裡可能是 list[list[tuple]]
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]
            unique = unique_words(words)'''
            link = base_url + a_tag['href']
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # 抓取文章內容
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content = article_soup.find_all('p', class_='MsoNormal')
            content_text = "\n".join(p.get_text(strip=True)  for p in content  if p.get_text(strip=True))

            found_special_terms = [term for term in special_terms if term in content_text]

            tags = tagger.tag(content_text)
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]

            for term in found_special_terms:
              for subword in term:
                  words = [word for word in words if word != subword]

            words += found_special_terms

            counter = Counter(words)
            top_10 = counter.most_common(10)
            print(f"{news_count}. {title}\n   {top_10}\n   {found_special_terms}\n")

            #print(f"{news_count}. {title}\n   {content_text}\n")
            #print(f"{news_count}. [{date}] {title}\n   {tags}\n   {unique}\n   👉 {link}\n")

    if not found_news:
        break  # 沒有新內容就停止
    page += 1


國家實驗研究院新聞列表：

1. 科教館「科學家的秘密基地」 4/16更新展區重新亮相
   [('中心', 17), ('海洋', 13), ('基地', 12), ('研究', 8), ('國家', 7), ('動物', 7), ('實驗', 6), ('參觀', 6), ('基因', 6), ('者', 6)]
   ['國家生物模式中心', '國家地震工程研究中心', '台灣海洋科技研究中心']

2. 【國研院說明稿】關於小油坑失火事件  若鑑定結果為國研院設備造成 國研院絕不卸責
   [('國研院', 8), ('中心', 5), ('陽明山', 3), ('國家', 3), ('表示', 3), ('國網', 3), ('設備', 3), ('調查', 3), ('空氣', 3), ('資料', 3)]
   ['國家高速網路與計算中心']

3. 【國研院說明稿】關於小油坑失火事件
   [('空氣', 6), ('品質', 6), ('感測器', 5), ('國家', 4), ('地區', 3), ('中心', 2), ('小油坑', 2), ('陽明山', 2), ('公園', 2), ('本', 2)]
   ['國家高速網路與計算中心']

4. 臺法電子資訊與器官晶片深化交流 合作提升雙方科技研發量能
   [('研究', 12), ('法國', 11), ('技術', 10), ('器官', 9), ('國家', 8), ('國研院', 7), ('合作', 7), ('晶片', 7), ('研究院', 5), ('交流', 5)]
   ['國家生物模式中心', '國家儀器科技研究中心']

5. 國立陽明交通大學、振興醫院與國研院國儀中心 共同開發新型微米級光纖麥克風 為聽障者提供更無礙的聲音感受
   [('麥克風', 13), ('光纖', 10), ('聲音', 7), ('中心', 7), ('國儀', 6), ('應用', 5), ('薄膜', 5), ('此', 4), ('新型', 4), ('微米級', 4)]
   ['國家儀器科技研究中心']

6. TAIWAN AI RAP試營運啟動 徵求試營運用戶，免費算力助攻AI開發！
   [('開發', 19), ('應用', 15), ('中心', 7), ('服務

## ckiptagger

In [ ]:
pip install -U ckiptagger[tf,gdown]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:23<00:00, 81.1MB/s]


In [ ]:
ws = WS("./data")
pos = POS("./data")

/usr/local/lib/python3.9/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [ ]:
sentence_list = [
    "綜合生魚片中的旗魚非常乾，有點難以下嚥，鮪魚有一點腥，其餘皆尚可，炒松阪肉味道調很好，肉質Q彈有嚼勁，蝦子給的算蠻豐盛蠻大隻的，但吃起來沒有很甜，肉有點老，炸白身魚外酥內軟，還有炸米餅，口感層次很多，很推一的一道菜",
    "點了皇家牛來的時候外觀真的是下一位。肥油沒處理筋一堆真的是不知道在吃什麼很反胃"
]

In [ ]:
word_sentence_list = ws(sentence_list)

In [ ]:
word_sentence_list

[['綜合',
  '生魚片',
  '中',
  '的',
  '旗魚',
  '非常',
  '乾',
  '，',
  '有點',
  '難以',
  '下嚥',
  '，',
  '鮪魚',
  '有',
  '一點',
  '腥',
  '，',
  '其餘',
  '皆',
  '尚可',
  '，',
  '炒',
  '松阪肉',
  '味道',
  '調',
  '很',
  '好',
  '，',
  '肉質',
  'Q彈',
  '有',
  '嚼勁',
  '，',
  '蝦子',
  '給',
  '的',
  '算',
  '蠻',
  '豐盛',
  '蠻',
  '大',
  '隻',
  '的',
  '，',
  '但',
  '吃起來',
  '沒有',
  '很',
  '甜',
  '，',
  '肉',
  '有點',
  '老',
  '，',
  '炸',
  '白身魚',
  '外酥',
  '內',
  '軟',
  '，',
  '還',
  '有',
  '炸米餅',
  '，',
  '口感',
  '層次',
  '很多',
  '，',
  '很',
  '推',
  '一',
  '的',
  '一',
  '道',
  '菜'],
 ['點',
  '了',
  '皇家',
  '牛',
  '來',
  '的',
  '時候',
  '外觀',
  '真的',
  '是',
  '下',
  '一',
  '位',
  '。',
  '肥油',
  '沒',
  '處理',
  '筋',
  '一',
  '堆',
  '真的',
  '是',
  '不',
  '知道',
  '在',
  '吃',
  '什麼',
  '很',
  '反胃']]

In [ ]:
pos_sentence_list = pos(word_sentence_list)
pos_sentence_list

[['A',
  'Na',
  'Ng',
  'DE',
  'Na',
  'Dfa',
  'VH',
  'COMMACATEGORY',
  'Dfa',
  'D',
  'VA',
  'COMMACATEGORY',
  'Na',
  'V_2',
  'Neqa',
  'VH',
  'COMMACATEGORY',
  'Neqa',
  'D',
  'VH',
  'COMMACATEGORY',
  'VC',
  'Na',
  'Na',
  'Na',
  'Dfa',
  'VH',
  'COMMACATEGORY',
  'Na',
  'Na',
  'V_2',
  'Na',
  'COMMACATEGORY',
  'Na',
  'VD',
  'DE',
  'VG',
  'Dfa',
  'VH',
  'Dfa',
  'VH',
  'Nf',
  'T',
  'COMMACATEGORY',
  'Cbb',
  'D',
  'D',
  'Dfa',
  'VH',
  'COMMACATEGORY',
  'Na',
  'Dfa',
  'VH',
  'COMMACATEGORY',
  'VC',
  'Na',
  'Na',
  'Ncd',
  'VH',
  'COMMACATEGORY',
  'D',
  'V_2',
  'Na',
  'COMMACATEGORY',
  'Na',
  'Na',
  'VH',
  'COMMACATEGORY',
  'Dfa',
  'VC',
  'Neu',
  'DE',
  'Neu',
  'Nf',
  'Na'],
 ['VC',
  'Di',
  'Na',
  'Na',
  'VA',
  'DE',
  'Na',
  'Na',
  'D',
  'SHI',
  'Nes',
  'Neu',
  'Nf',
  'PERIODCATEGORY',
  'Na',
  'D',
  'VC',
  'Na',
  'Neu',
  'Nf',
  'D',
  'SHI',
  'D',
  'VK',
  'D',
  'VC',
  'Nep',
  'Dfa',
  'VH']]

In [ ]:
# https://www.nltk.org/
import nltk
import json
from google.colab import files # 在 Google Colab 環境上傳檔案時所用；若在個人電腦執行 Python 則不需要

nltk.download('punkt')

In [ ]:
# 若下載 python 檔並在自己電腦執行，開檔讀檔請用這段 # 若要上傳檔案到 Google Colab 虛擬主機，請註解掉這段
#f = open("D:/cjlin/sample/ReutersCorn-sample10.json", "r", encoding='UTF-8') # encoding='UTF-8' 才能正確讀入中文檔案
#docText = f.read()
#f.close()

In [ ]:
# 由電腦上傳檔案至 Colab 虛擬主機 # 若要在個人電腦單機作業，請註解掉這段
file = files.upload()
trf = open("ReutersCorn-train.json", "r", encoding='UTF-8')
fdoc = trf.read()
trf.close()

In [ ]:
dataset = json.loads(fdoc)
dataset[0]['text'] # 此行僅用來確認執行結果正確，可刪去

In [ ]:
file = files.upload()
dictf = open("posDict_ReutersCorn.json", "r", encoding='UTF-8')
dict_raw = dictf.read()
dictf.close()
POSdict = json.loads(dict_raw)

In [ ]:
wordTotal = 0
tokenSet = set()
for doc in dataset:
  word = nltk.word_tokenize(doc['text'])
  wordTotal += len(word)
  #nltk_token = []
  for w in word:
    tokenSet.add(w)
    #nltk_token.append(w)
  #doc['nltk_token'] = nltk_token
print(wordTotal)
len(tokenSet)

In [ ]:
lowerSet = set()
for w in tokenSet:
  lowerSet.add(w.lower())
len(lowerSet)

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('omw-1.4')

In [ ]:
word_sets = ['automate', 'automatic', 'automation', 'computer', 'computerize', 'computerization', 'computerizational']
for w in word_sets:
  stemstr = lemmatizer.lemmatize(w, "v")
  print(w + ' ==> ' + stemstr)

In [ ]:
lemmaWOpos = set()
for w in lowerSet:
  lemmaWOpos.add(lemmatizer.lemmatize(w))
len(lemmaWOpos)

In [ ]:
lemmaWpos = set()
for w in lowerSet:
  pos = POSdict.get(w)
  if pos != None and pos != "x":
    if pos == "j": pos = "a"
    lemmaWpos.add(lemmatizer.lemmatize(w, pos))
  else:
    lemmaWpos.add(lemmatizer.lemmatize(w))
len(lemmaWpos)

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
stemSet = set()
for w in lowerSet:
  stemSet.add(stemmer.stem(w))
len(stemSet)

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
word_sets = ['automate', 'automatic', 'automation', 'computer', 'computerize', 'computerization', 'computerizational']
for w in word_sets:
  stemstr = stemmer.stem(w)
  print(w + ' ==> ' + stemstr)


In [ ]:
pip install -U ckiptagger[tf,gdown]

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
data_utils.download_data_gdown("./")

In [ ]:
ws = WS("./data")
pos = POS("./data")

In [ ]:
sentence_list = [
    "土地公有政策?？還是土地婆有政策。.",
    "某某候選人提出的土地公有政策",
    "最多容納59,000個人,或5.9萬人,再多就不行了.這是環評的結論.",
    "電子計算機是會計算題目的機器。",
]


In [ ]:
sent_seg = ws(["高高興興打打球,丟丟看成績如何、開不開心"])

In [ ]:
sent_seg[0]

In [ ]:
sent_pos = pos(sent_seg)
sent_pos[0]